In [73]:
import pandas as pd

In [74]:
routes = pd.read_csv("jyska_routes_2024-06-04.csv")
aq = pd.read_csv("Jyväskylä Jyskä 5.3.2024 - 4.6.2024_7845726d-005e-4a28-925d-f8aa76982d2a.csv")

In [75]:
import numpy as np
# Muunetaan "-" NaN arvoksi
aq = aq.replace('-',np.NaN)
aq["Date"] = aq["Vuosi"].astype(str) + "-" + aq["Kuukausi"].astype(str) + "-" +  aq["Päivä"].astype(str) + " " + aq["Aika [UTC]"]
aq["Date"] = pd.to_datetime(aq["Date"])
aq["Hengitettävät hiukkaset <10 µm [µg/m3]"] = aq["Hengitettävät hiukkaset <10 µm [µg/m3]"].astype(float)
aq["Pienhiukkaset <2.5 µm [µg/m3]"] = aq["Pienhiukkaset <2.5 µm [µg/m3]"].astype(float)
aq.dtypes

Havaintoasema                                     object
Vuosi                                              int64
Kuukausi                                           int64
Päivä                                              int64
Aika [UTC]                                        object
Hengitettävät hiukkaset <10 µm [µg/m3]           float64
Pienhiukkaset <2.5 µm [µg/m3]                    float64
Date                                      datetime64[ns]
dtype: object

In [76]:
import datetime
import warnings
warnings.filterwarnings('ignore')


routes["endTime"] = pd.to_datetime(routes["endTime"])
routes["startTime"] = pd.to_datetime(routes["startTime"])
duration_hours = (routes["endTime"] - routes["startTime"]).dt.total_seconds() / 3600

duration_by_day = duration_hours.groupby(routes['endTime'].dt.to_period('d')).sum()
duration_by_hour = duration_hours.groupby(routes['endTime'].dt.to_period('h')).sum()

durations = duration_hours.groupby(routes['endTime']).sum()
duration_by_day.index = duration_by_day.index.to_timestamp()
duration_by_hour.index = duration_by_hour.index.to_timestamp()
duration_by_day.name = "brushing"

In [77]:
#Päivittäiset keskiarvot
filter_df = aq[["Date","Hengitettävät hiukkaset <10 µm [µg/m3]", "Pienhiukkaset <2.5 µm [µg/m3]"]]
filter_df = filter_df.resample('D', on='Date').mean().reset_index()
merge_df = filter_df.merge(duration_by_day, left_on="Date", right_index=True, how="left")


merge_df["brushing"] = merge_df["brushing"].fillna(0)

merge_df["boolean_brushing"] = merge_df['brushing'].astype(bool).astype(int)

In [78]:
no_brushing = merge_df[merge_df["boolean_brushing"] == 0]
print(f"Ei harjausta: {no_brushing['Hengitettävät hiukkaset <10 µm [µg/m3]'].mean()}")
yes_brushing = merge_df[merge_df["boolean_brushing"] > 0]
print(f"Harjausta: {yes_brushing['Hengitettävät hiukkaset <10 µm [µg/m3]'].mean()}")
print(f"Harjauspäivien määrä: {len(yes_brushing)}")

Ei harjausta: 16.02360033259424
Harjausta: 21.22416666666667
Harjauspäivien määrä: 10


In [79]:
merge_df['previous_day_brushing'] = merge_df["boolean_brushing"].shift(1)


# Filter DataFrame for days with rain and days without rain on the previous day
brushing_previous_day = merge_df[merge_df['previous_day_brushing'] > 0]
no_brushing_previous_day = merge_df[merge_df['previous_day_brushing'] == 0]

# Calculate mean air quality for each group
mean_air_quality_brushing = brushing_previous_day['Hengitettävät hiukkaset <10 µm [µg/m3]'].mean()
mean_air_quality_non_brushing = no_brushing_previous_day['Hengitettävät hiukkaset <10 µm [µg/m3]'].mean()


print("Ei harjausta edellisenä päivänä:", mean_air_quality_brushing)
print("Harjausta edellisenä päivänä:", mean_air_quality_non_brushing)

Ei harjausta edellisenä päivänä: 23.339166666666667
Harjausta edellisenä päivänä: 15.856659184436964


In [81]:
# TODO: Tuntikorrelaatio